Here is an implementation of the Byte-Pair Encoding Compression Algorithm (which is out of the scope of the book). This BPE tokenization method is used to train the GPT models.

In [37]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
corpus=[
    "The cat sat quietly by the window, watching the birds outside.",
    "She enjoys reading books on sunny afternoons in the park.",
    "He picked fresh apples from the orchard and shared them with friends.",
    "A warm breeze drifted through the open door as the sun set.",
    "The children played games and laughed together on the grassy field."
]

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [40]:
word_freq = {}
for sentence in corpus:
    words_with_offset=tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(sentence)
    new_words=[word for word,offset in words_with_offset]
    for word in new_words:
        word_freq[word] = 1 + word_freq.get(word,0)
print(word_freq)

{'The': 2, 'Ġcat': 1, 'Ġsat': 1, 'Ġquietly': 1, 'Ġby': 1, 'Ġthe': 7, 'Ġwindow': 1, ',': 1, 'Ġwatching': 1, 'Ġbirds': 1, 'Ġoutside': 1, '.': 5, 'She': 1, 'Ġenjoys': 1, 'Ġreading': 1, 'Ġbooks': 1, 'Ġon': 2, 'Ġsunny': 1, 'Ġafternoons': 1, 'Ġin': 1, 'Ġpark': 1, 'He': 1, 'Ġpicked': 1, 'Ġfresh': 1, 'Ġapples': 1, 'Ġfrom': 1, 'Ġorchard': 1, 'Ġand': 2, 'Ġshared': 1, 'Ġthem': 1, 'Ġwith': 1, 'Ġfriends': 1, 'A': 1, 'Ġwarm': 1, 'Ġbreeze': 1, 'Ġdrifted': 1, 'Ġthrough': 1, 'Ġopen': 1, 'Ġdoor': 1, 'Ġas': 1, 'Ġsun': 1, 'Ġset': 1, 'Ġchildren': 1, 'Ġplayed': 1, 'Ġgames': 1, 'Ġlaughed': 1, 'Ġtogether': 1, 'Ġgrassy': 1, 'Ġfield': 1}


In [41]:
alphabet=[]

for word in word_freq:
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

vocab = ["<|endoftext|>"] + alphabet.copy()
print(vocab)

['<|endoftext|>', ',', '.', 'A', 'H', 'S', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'y', 'z', 'Ġ']


In [42]:
splits = {word: [c for c in word] for word in word_freq.keys()}
print(splits)

{'The': ['T', 'h', 'e'], 'Ġcat': ['Ġ', 'c', 'a', 't'], 'Ġsat': ['Ġ', 's', 'a', 't'], 'Ġquietly': ['Ġ', 'q', 'u', 'i', 'e', 't', 'l', 'y'], 'Ġby': ['Ġ', 'b', 'y'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'Ġwindow': ['Ġ', 'w', 'i', 'n', 'd', 'o', 'w'], ',': [','], 'Ġwatching': ['Ġ', 'w', 'a', 't', 'c', 'h', 'i', 'n', 'g'], 'Ġbirds': ['Ġ', 'b', 'i', 'r', 'd', 's'], 'Ġoutside': ['Ġ', 'o', 'u', 't', 's', 'i', 'd', 'e'], '.': ['.'], 'She': ['S', 'h', 'e'], 'Ġenjoys': ['Ġ', 'e', 'n', 'j', 'o', 'y', 's'], 'Ġreading': ['Ġ', 'r', 'e', 'a', 'd', 'i', 'n', 'g'], 'Ġbooks': ['Ġ', 'b', 'o', 'o', 'k', 's'], 'Ġon': ['Ġ', 'o', 'n'], 'Ġsunny': ['Ġ', 's', 'u', 'n', 'n', 'y'], 'Ġafternoons': ['Ġ', 'a', 'f', 't', 'e', 'r', 'n', 'o', 'o', 'n', 's'], 'Ġin': ['Ġ', 'i', 'n'], 'Ġpark': ['Ġ', 'p', 'a', 'r', 'k'], 'He': ['H', 'e'], 'Ġpicked': ['Ġ', 'p', 'i', 'c', 'k', 'e', 'd'], 'Ġfresh': ['Ġ', 'f', 'r', 'e', 's', 'h'], 'Ġapples': ['Ġ', 'a', 'p', 'p', 'l', 'e', 's'], 'Ġfrom': ['Ġ', 'f', 'r', 'o', 'm'], 'Ġorchard': ['Ġ', 'o'

In [43]:
def compute_pair_freq(splits):
    pair_freqs={}
    for word,freq in word_freq.items():
        split=splits[word]
        if len(split)==1:
            continue
        for i in range(len(split)-1):
            pair=(split[i],split[i+1])
            pair_freqs[pair] = freq + pair_freqs.get(pair,0)
    return pair_freqs

In [44]:
pair_freqs = compute_pair_freq(splits)
print(pair_freqs)

{('T', 'h'): 2, ('h', 'e'): 13, ('Ġ', 'c'): 2, ('c', 'a'): 1, ('a', 't'): 3, ('Ġ', 's'): 5, ('s', 'a'): 1, ('Ġ', 'q'): 1, ('q', 'u'): 1, ('u', 'i'): 1, ('i', 'e'): 3, ('e', 't'): 3, ('t', 'l'): 1, ('l', 'y'): 1, ('Ġ', 'b'): 4, ('b', 'y'): 1, ('Ġ', 't'): 10, ('t', 'h'): 11, ('Ġ', 'w'): 4, ('w', 'i'): 2, ('i', 'n'): 4, ('n', 'd'): 4, ('d', 'o'): 2, ('o', 'w'): 1, ('w', 'a'): 2, ('t', 'c'): 1, ('c', 'h'): 3, ('h', 'i'): 2, ('n', 'g'): 2, ('b', 'i'): 1, ('i', 'r'): 1, ('r', 'd'): 2, ('d', 's'): 2, ('Ġ', 'o'): 5, ('o', 'u'): 2, ('u', 't'): 1, ('t', 's'): 1, ('s', 'i'): 1, ('i', 'd'): 1, ('d', 'e'): 1, ('S', 'h'): 1, ('Ġ', 'e'): 1, ('e', 'n'): 4, ('n', 'j'): 1, ('j', 'o'): 1, ('o', 'y'): 1, ('y', 's'): 1, ('Ġ', 'r'): 1, ('r', 'e'): 5, ('e', 'a'): 1, ('a', 'd'): 1, ('d', 'i'): 1, ('b', 'o'): 1, ('o', 'o'): 3, ('o', 'k'): 1, ('k', 's'): 1, ('o', 'n'): 3, ('s', 'u'): 2, ('u', 'n'): 2, ('n', 'n'): 1, ('n', 'y'): 1, ('Ġ', 'a'): 5, ('a', 'f'): 1, ('f', 't'): 2, ('t', 'e'): 2, ('e', 'r'): 2, ('r', 

In [45]:
best_pair =""
max_freq = -1

for pair,freq in pair_freqs.items():
    if max_freq<freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('h', 'e') 13


In [46]:
merges = {("h","e"):'he'}
vocab.append('he')

In [47]:
def merge_pairs(a,b,splits):
    for word in word_freq:
        split = splits[word]
        if len(split)==1:
            continue

        i=0
        while i<len(split)-1:
            if split[i]==a and split[i+1]==b:
                split=split[:i] + [a+b] + split[i+2:]
            else:
                i+=1
        splits[word] = split
    return splits

In [48]:
splits = merge_pairs("h","e",splits)
print(splits)

{'The': ['T', 'he'], 'Ġcat': ['Ġ', 'c', 'a', 't'], 'Ġsat': ['Ġ', 's', 'a', 't'], 'Ġquietly': ['Ġ', 'q', 'u', 'i', 'e', 't', 'l', 'y'], 'Ġby': ['Ġ', 'b', 'y'], 'Ġthe': ['Ġ', 't', 'he'], 'Ġwindow': ['Ġ', 'w', 'i', 'n', 'd', 'o', 'w'], ',': [','], 'Ġwatching': ['Ġ', 'w', 'a', 't', 'c', 'h', 'i', 'n', 'g'], 'Ġbirds': ['Ġ', 'b', 'i', 'r', 'd', 's'], 'Ġoutside': ['Ġ', 'o', 'u', 't', 's', 'i', 'd', 'e'], '.': ['.'], 'She': ['S', 'he'], 'Ġenjoys': ['Ġ', 'e', 'n', 'j', 'o', 'y', 's'], 'Ġreading': ['Ġ', 'r', 'e', 'a', 'd', 'i', 'n', 'g'], 'Ġbooks': ['Ġ', 'b', 'o', 'o', 'k', 's'], 'Ġon': ['Ġ', 'o', 'n'], 'Ġsunny': ['Ġ', 's', 'u', 'n', 'n', 'y'], 'Ġafternoons': ['Ġ', 'a', 'f', 't', 'e', 'r', 'n', 'o', 'o', 'n', 's'], 'Ġin': ['Ġ', 'i', 'n'], 'Ġpark': ['Ġ', 'p', 'a', 'r', 'k'], 'He': ['H', 'e'], 'Ġpicked': ['Ġ', 'p', 'i', 'c', 'k', 'e', 'd'], 'Ġfresh': ['Ġ', 'f', 'r', 'e', 's', 'h'], 'Ġapples': ['Ġ', 'a', 'p', 'p', 'l', 'e', 's'], 'Ġfrom': ['Ġ', 'f', 'r', 'o', 'm'], 'Ġorchard': ['Ġ', 'o', 'r', 'c', 

In [ ]:
print(merges)

In [49]:
vocab_size=50

while len(vocab)<vocab_size:
    pair_freqs = compute_pair_freq(splits)
    best_pair=""
    max_freq=-1
    for pair,freq in pair_freqs.items():
        if freq>max_freq:
            max_freq=freq
            best_pair=pair
    splits = merge_pairs(*best_pair,splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0]+best_pair[1])

In [32]:
print(vocab)

['<|endoftext|>', ',', '.', 'A', 'H', 'S', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'y', 'z', 'Ġ', 'he', 'Ġt', 'Ġthe', 'Ġs', 'Ġo', 're', 'Ġa', 'Ġb', 'Ġw', 'in', 'Ġf', 'at', 'ie', 'ch', 'oo', 'Ġp', 'ar', 'ed']


In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word,offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair,merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i<len(split)-1:
                if split[i]==pair[0] and split[i+1]==pair[1]:
                    split = split[:i]+[merge]+split[i+2:]
                else:
                    i+=1
            splits[idx]=split
    return sum(splits,[])

In [36]:
print(tokenize("The cat sat by the window."))

['T', 'he', 'Ġ', 'c', 'at', 'Ġs', 'at', 'Ġb', 'y', 'Ġthe', 'Ġw', 'in', 'd', 'o', 'w', '.']
